<a href="https://colab.research.google.com/github/mathu3004/Pearl_Path/blob/Personalized_Itinerary_Generator_Based_Radius/Model_Training_for_Travel_Planner_Based_Radius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, MultiLabelBinarizer
from geopy.geocoders import Nominatim
from urllib.parse import urlparse

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Restaurants Data Preprocessed
# Load the merged CSV file
file_path = '/content/drive/My Drive/Colombo/FinalPreprocessedMergedRestaurants.csv'
df = pd.read_csv(file_path)

# 1. Address Preprocessing
# Geocode to latitude and longitude (if needed)
geolocator = Nominatim(user_agent="geoapiExercises")
def get_coordinates(address):
    try:
        location = geolocator.geocode(address)
        return location.latitude, location.longitude
    except:
        return None, None

# Uncomment if geocoding is needed
#df['latitude'], df['longitude'] = zip(*df['address'].apply(get_coordinates))

# 2. Category Encoding
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])

'''# 3. Review and Ranking
df['numberofreviews'] = df['numberofreviews'].fillna(0)
df['rankingposition'] = df['rankingposition'].fillna(df['rankingposition'].median())
df['rankingdenominator'] = df['rankingdenominator'].fillna(1)

# Calculate ranking percentage
df['ranking_percentage'] = df['rankingposition'] / df['rankingdenominator']

# Extract numerical data from ranking string
df['ranking_position'] = df['rankingstring'].str.extract(r'#(\d+)').astype(float)
df['ranking_total'] = df['rankingstring'].str.extract(r'of (\d+)').astype(float)
df['ranking_normalized'] = df['ranking_position'] / df['ranking_total']

# 4. Rating Normalization
scaler = MinMaxScaler()
df['rating'] = df['rating'].fillna(df['rating'].median())
df['rawranking'] = df['rawranking'].fillna(df['rawranking'].median())
df[['rating', 'rawranking']] = scaler.fit_transform(df[['rating', 'rawranking']])'''

# 5. Multi-Label Columns Preprocessing
def process_list_columns(column):
    return column.apply(lambda x: [item.strip() for item in str(x).split(',')] if pd.notnull(x) else [])

# Step 1: Create a mapping dictionary for city grouping
city_mapping = {
    # Map to Nuwara Eliya
    'Kotmale': 'Nuwara Eliya',
    'Nanu Oya': 'Nuwara Eliya',
    'Ramboda': 'Nuwara Eliya',
    'Talawakele': 'Nuwara Eliya',
    'Ambewela': 'Nuwara Eliya',

    # Map to Ella
    'Haputale': 'Ella',
    'Bandarawela': 'Ella',
    'Kithalella': 'Ella',
    'Wellawaya': 'Ella',

    # Map to Kandy
    'Peradeniya': 'Kandy',
    'Gampola': 'Kandy',
    'Kadugannawa': 'Kandy',
    'Tennekumbura': 'Kandy',

    # Map to Colombo
    'Kadawata': 'Colombo',
    'Maharagama': 'Colombo',
    'Wattala': 'Colombo',
    'Kaduwela': 'Colombo',
    'Rajagiriya': 'Colombo',
    'Battaramulla': 'Colombo',
    'Dehiwala-Mount Lavinia': 'Colombo',
    'Nugegoda': 'Colombo',
    'Mount Lavinia': 'Colombo',
    'Uswetakeiyawa': 'Colombo',
    'Boralesgamuwa': 'Colombo',
    'Malabe': 'Colombo',
    'Thalawathugoda': 'Colombo',
    'Dehiwala': 'Colombo',
    'Kelaniya': 'Colombo',
    'Kiribathgoda': 'Colombo',
    'Panadura': 'Colombo'
}

# Apply the city mapping to create a permanent city column
df['city'] = df['addressobj_city'].replace(city_mapping)

# Ensure all other cities are labeled correctly
df['city'] = df['city'].apply(lambda x: x if x in ['Nuwara Eliya', 'Ella', 'Kandy', 'Colombo'] else 'Other')

# 3. One-Hot Encoding to create only four main city columns
city_dummies = pd.get_dummies(df['city'], prefix='city')[['city_Colombo', 'city_Ella', 'city_Kandy', 'city_Nuwara Eliya']].astype(int)
df = pd.concat([df, city_dummies], axis=1)

# drop addressobj_city column
df.drop(columns=['addressobj_city'], inplace=True)

df['cuisines'] = process_list_columns(df['cuisines'])
df['features'] = process_list_columns(df['features'])
df['dietaryrestrictions'] = process_list_columns(df['dietaryrestrictions'])
df['mealtypes'] = process_list_columns(df['mealtypes'])

# One-Hot Encoding for multi-label data
mlb = MultiLabelBinarizer()
# Create DataFrames for one-hot encoded features and add prefix to column names
cuisine_df = pd.DataFrame(mlb.fit_transform(df['cuisines']), columns=mlb.classes_, index=df.index)
cuisine_df = cuisine_df.add_prefix('cuisine_')  # Add prefix here
df = df.join(cuisine_df)

feature_df = pd.DataFrame(mlb.fit_transform(df['features']), columns=mlb.classes_, index=df.index)
feature_df = feature_df.add_prefix('feature_')  # Add prefix here
df = df.join(feature_df)

dietary_df = pd.DataFrame(mlb.fit_transform(df['dietaryrestrictions']), columns=mlb.classes_, index=df.index)
dietary_df = dietary_df.add_prefix('dietary_')  # Add prefix here
df = df.join(dietary_df)

mealtype_df = pd.DataFrame(mlb.fit_transform(df['mealtypes']), columns=mlb.classes_, index=df.index)
mealtype_df = mealtype_df.add_prefix('mealtype_')  # Add prefix here
df = df.join(mealtype_df)

# Save the final cleaned data to a new file
final_processed_file_path = "/content/drive/My Drive/Colombo/LastPreprocessedMergedRestaurants.csv"
df.to_csv(final_processed_file_path, index=False)

In [ ]:
# User Data Preprocessing
# Load the uploaded file
file_path = "/content/drive/My Drive/DataPre/preprocessed_user_inputs.csv"
df = pd.read_csv(file_path)

# Generate synthetic 'maximum_distance' column with values ranging from 10km to 50km
df['maximum_distance'] = np.random.randint(10, 51, size=len(df))

# Save the updated file back to Google Drive
file_path = "/content/drive/My Drive/DataPre/preprocessed_user_inputs.csv"
df.to_csv(file_path, index=False)

print("File successfully saved!")